In [2]:
!pip install torch
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.4/645.4 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [3]:
import os
import math
import cv2
import torch
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt

In [4]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# setting the train and evaluation dataset directories. Change the below to your own path
root_path = '/content/drive/MyDrive/CS4243 Project/data/'
train_path = '/content/drive/MyDrive/CS4243 Project/data/frames/train'
# test_path = '/content/drive/MyDrive/CS4243 Project/data/frames/test'

In [6]:
class_labels = [name for name in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, name))]
print(class_labels)
# Uncomment to clean a specific folder
# class_labels = ["normal"]

clean_train_path = os.path.join(root_path, "image_data_cleaned")

['norm', 'weap']


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
def save_image(image, image_name, class_label):
    save_path = os.path.join(clean_train_path, class_label, image_name)
    cv2.imwrite(save_path, image)

# crop by expanding bounding box by 10% of image size
crop_expansion_factor = 0.1

def detect_and_crop_person(image_names, class_label):
    image_paths = [os.path.join(train_path, class_label, name) for name in image_names]
    results = model(image_paths)
    detections = results.pandas().xyxy
    success = 0
    for i, df in enumerate(detections):
        if "person" in df.name.values:
            rows = df[df.name == "person"]
            min_values = rows.min()
            max_values = rows.max()
            row_ymin = min_values["ymin"]
            row_ymax = max_values["ymax"]
            row_xmin = min_values["xmin"]
            row_xmax = max_values["xmax"]
            image = cv2.imread(image_paths[i])
            height, width, _ = image.shape
            ymin = max(0, int(row_ymin - height * crop_expansion_factor))
            ymax = min(height, int(row_ymax + height * crop_expansion_factor))
            xmin = max(0, int(row_xmin - width * crop_expansion_factor))
            xmax = min(width, int(row_xmax + width * crop_expansion_factor))
            cropped = image[ymin:ymax, xmin:xmax]
            success += 1
            save_image(cropped, image_names[i], class_label)
    # Return number of dropped examples
    return len(image_names) - success

In [ ]:
# larger is faster, but uses more memory
batch_size = 128

for label in class_labels:
    print("Cleaning images from", label)
    input_path = os.path.join(train_path, label)
    output_path = os.path.join(clean_train_path, label)
    Path(output_path).mkdir(parents=True, exist_ok=True)
    image_names = [name for name in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, name))]
    image_num = len(image_names)
    print("Found", image_num, "images to be processed in", math.ceil(image_num / batch_size), "steps")
    image_name_batches = [image_names[i:i+batch_size] for i in range(0, image_num, batch_size)]
    total_processed = 0
    total_dropped = 0
    for i, name_batch in enumerate(image_name_batches):
        total_dropped += detect_and_crop_person(name_batch, label)
        total_processed += batch_size
        print(f"Step {i} | Processed: {total_processed} | Dropped: {total_dropped}")